In [1]:
# !uv pip install ipyykernel

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import sentence_transformers
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from tqdm import tqdm
from langchain.llms import Together
from langchain.prompts import PromptTemplate
import textwrap

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [199]:
import time
from functools import wraps

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Function '{func.__name__}' executed in {end - start:.4f} seconds")
        return result
    return wrapper

In [4]:
embedding = HuggingFaceEmbeddings(model="BAAI/bge-large-en-v1.5" ,encode_kwargs={"normalize_embeddings": True})
db = FAISS.load_local("faiss_index_updated", embedding,allow_dangerous_deserialization=True)

In [2]:
from langchain.document_loaders import TextLoader

loader = TextLoader("output_tex1.txt")
documents = loader.load()
documents[0].page_content

'\n--- Page 1 ---\n431278/202\n1\nNATIONAL HIGHWAYS LOGISTICS MANAGEMENT LIMITED(formerly known as COCHIN PORT ROAD COMPANY LIMITED)Regd. Office: NHAI HQ, G-5&6, Sector-10, Dwarka, New Delhi-110 075Email id: spvsecretarial@nhai ,orq CIN: U63000DL2004PLC124173 Phn: 011-25074100\nCERTIFIED TRUE COPY OF THE RESOLUTION PASSED IN THE BOARD MEETINGOF NATIONAL HIGHWAYS LOGISTICS MANAGEMENT LIMITED” (formerly known asCOCHIN PORT ROAD COMPANY LIMITED) HELD ON TUESDAY, 29TH MARCH, 2022AT 01 :00 PM AND CONCLUDED AT 03:45 P.M. AT THE REGISTERED OFFICE OF THECOMPANY SITUATED AT NHAI-HQ, G-5 & 6, SECTOR-10, DWARKA, NEW DELHI-110075.To consider and approve Delegation of Power.Delegation of Power for financial as well as administrative matters were presented to theBoard.Board discussed the same in detail and with certain modifications approved the delegationof power. The approved delegation of power is placed at Annexure-2Certified true copyFor and on behalf of Board of Directors ofNATIONAL HIGHWAYS L

In [3]:
splitter = CharacterTextSplitter(chunk_size=600,chunk_overlap=100, separator= '.')
docs = splitter.split_documents(documents)

Created a chunk of size 613, which is longer than the specified 600
Created a chunk of size 864, which is longer than the specified 600
Created a chunk of size 689, which is longer than the specified 600
Created a chunk of size 691, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 613, which is longer than the specified 600
Created a chunk of size 712, which is longer than the specified 600
Created a chunk of size 610, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 604, which is longer than the specified 600
Created a chunk of size 679, which is longer than the specified 600
Created a chunk of size 820, which is longer than the specified 600
Created a chunk of size 960, which is longer than the specified 600
Created a chunk of size 609, which is longer than the specified 600
Created a chunk of size 1052, which is longer th

In [6]:
embeddings = HuggingFaceEmbeddings(model="BAAI/bge-large-en-v1.5" ,encode_kwargs={"normalize_embeddings": True}) #thenlper/gte-small 
db = FAISS.from_documents(docs, embeddings)


_ = [embeddings.embed_query(doc.page_content) for doc in tqdm(docs)]

100%|██████████| 211/211 [04:41<00:00,  1.33s/it]


In [21]:
db.save_local("faiss_index_updated")

In [181]:
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
import os

load_dotenv()
api_key = os.getenv("together_api_key")

In [13]:
llm = Together(
    model="lgai/exaone-3-5-32b-instruct",
    temperature=0,
    together_api_key=api_key
)

In [252]:
retriever = db.as_retriever(search_kwargs={"k": 5, "fetch_k": 10})
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an intelligent and professional HR assistant for a corporate organization named National Highways Logistics Management Ltd (NHLML) and you are named as Tara.
You are talking to the employee of NHLML about whom you don't know anything except this.

Your responsibilities include:
- Answering the question with the help of the policy.
- Do not answer questions which are not related to HR or company policies.(e.g., food, pets, travel, entertainment,or any such casual request etc.).
- If the input is just a greeting or acknowledgment (e.g., "hi", "thanks", "okay"), respond politely without referencing any documents or context.
- If you detect spelling errors in the user's question and ONLY if the user's question is in english, first ask "Did you mean [corrected spelling]?" and wait for confirmation before providing the answer.
- You are supposed to refer to the whole context first and refer to chat history ONLY if needed. DO NOT refer to the chat history if it is not needed.
- Never tell the user to refer to the policy as you are already provided with it and are supposed to answer according to it.

Context:
{context}

Question:
{question}


Answer:
"""
)

In [268]:
intent_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
You are an intent classification assistant.

Classify the user's message into one of two categories:

- "ack" → If the message is ONLY a greeting, thanks, acknowledgment, or generic affirmation (e.g. "hi", "thanks", "okay", "great").
- "real" → If the message includes a real question or request (even if it starts with a greeting).

Message: "{query}"

Respond with only "ack" or "real".
"""
)

In [307]:
ack_response_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
You are Tara, a professional but friendly HR assistant at National Highways Logistics Management Ltd (NHLML).

The user has sent a greeting, acknowledgment, or polite message. Your job is to respond appropriately — warmly, professionally, and in a way that reflects what they said.
For Example:
- If the user says "thanks", respond with a natural variation like "You're very welcome!" or "Glad I could help!"
- If they say "hi" or "hello", greet them back and ask how you can assist.
- If they say "okay", "cool", "great", etc., acknowledge it appropriately without sounding robotic.
- Your response must feel like a human reply to what they actually said.

User said:
"{query}"

Tara's reply:
"""
)

ack_chain = LLMChain(llm=llm, prompt=ack_response_prompt)


In [326]:
# ConversationBufferWindowMemory(k=1)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    k=5
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  
    retriever=retriever,
    combine_docs_chain_kwargs={"prompt": prompt},
    memory=memory
 )


# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=False,
#     chain_type_kwargs={"prompt": prompt}
# )

In [309]:
from langchain import LLMChain
intent_chain = LLMChain(llm=llm, prompt=intent_prompt)

In [339]:
qa_chain.memory.clear()

In [347]:
query = "dahi bhalle ki recipe btao"

intent = intent_chain.run(query=query).strip().lower()

if intent == "ack":
    ack_response = ack_chain.run(query)
    # qa_chain.memory.clear()  # Optional: reset memory
    print("response", ack_response)
else:
    response = qa_chain.run(query)
    print("Bot:", response)
print("\n________Memory________:\n", memory.buffer)

Bot: I'm sorry, but I can't assist with that request as it falls outside the scope of HR-related inquiries or company policies. If you have any questions related to policies, benefits, or other HR matters at NHLML, feel free to ask!

________Memory________:
 [HumanMessage(content='kya haal hai', additional_kwargs={}, response_metadata={}), AIMessage(content='Namaste! How can I assist you today with regards to your queries or concerns at NHLML?', additional_kwargs={}, response_metadata={}), HumanMessage(content='can i get maternity leave if im a pregnant male', additional_kwargs={}, response_metadata={}), AIMessage(content='The current policies of National Highways Logistics Management Ltd (NHLML) specifically outline maternity leave for female executives, particularly those with less than two surviving children, for a period of up to 180 days from the date of commencement, subject to a medical certificate. There is no provision in the given policies for maternity leave for male individ